In [14]:
!pip install underthesea tensorflow scikit-learn pandas -q

In [15]:
import pandas as pd
import numpy as np
from underthesea import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical
import pickle
import warnings

In [16]:
VOCAB_SIZE = 15000      # Giới hạn số lượng từ trong từ điển
MAX_LENGTH = 150        # Độ dài tối đa của một câu (sau khi padding)
EMBEDDING_DIM = 128     # Số chiều của vector biểu diễn từ
NUM_EPOCHS = 10         # Số lần lặp qua toàn bộ dữ liệu
BATCH_SIZE = 32         # Số mẫu dữ liệu cho mỗi lần cập nhật trọng số

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# ===================================================================
# BƯỚC 1: TẢI VÀ CHUẨN BỊ DỮ LIỆU
# ===================================================================
print("--- [CNN-LSTM] Bắt đầu tải và chuẩn bị dữ liệu ---")

# Đọc file CSV
df = pd.read_csv('/content/drive/MyDrive/AI/Model/momo_review_clean_balanced.csv')

# Giữ lại cột nội dung và nhãn
df = df[['Noi_dung_sach_giu_dau', 'Nhãn']].dropna().reset_index(drop=True)

# Đổi tên cột cho dễ dùng
df.rename(columns={'Noi_dung_sach_giu_dau': 'text', 'Nhãn': 'label_text'}, inplace=True)

# Ánh xạ nhãn sang số
label_map = {"Tiêu cực": 0, "Trung lập": 1, "Tích cực": 2}
id_to_label = {v: k for k, v in label_map.items()}
df['label'] = df['label_text'].map(label_map)

print(df.head())


--- [CNN-LSTM] Bắt đầu tải và chuẩn bị dữ liệu ---
                                                text label_text  label
0  cốt chuyện bình thường mạch phim dài lê thể ch...   Tiêu cực      0
1  phim rất hay từ đầu đến cuối lôi cuốn và hấp d...   Tích cực      2
2            toàn nói chuyện không có cảnh đánh nhau  Trung lập      1
3  phim hay về nội dung lẫn về cảnh quay nhưng kh...   Tích cực      2
4  phim hay có nhiều tình tiết hấp dẫn có một vài...   Tích cực      2


In [20]:
# ===================================================================
# BƯỚC 2: CHIA DỮ LIỆU VÀ TIỀN XỬ LÝ CHO DEEP LEARNING
# ===================================================================
print("--- [CNN-LSTM] Chia dữ liệu và tiền xử lý ---")
X = df['text'] # Dùng cột 'text' chưa tách từ để Tokenizer của Keras tự xử lý
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Tokenizer: Xây dựng từ điển và chuyển văn bản thành chuỗi số nguyên
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding: Đảm bảo các chuỗi có cùng độ dài
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LENGTH, padding='post', truncating='post')

# One-hot encoding cho nhãn
y_train_cat = to_categorical(y_train, num_classes=3)
y_test_cat = to_categorical(y_test, num_classes=3)
print("Hoàn thành.")
print("-" * 30)

--- [CNN-LSTM] Chia dữ liệu và tiền xử lý ---
Hoàn thành.
------------------------------


In [21]:
# ===================================================================
# BƯỚC 3: XÂY DỰNG MÔ HÌNH KẾT HỢP CNN-LSTM
# (Đây là phần thay đổi chính)
# ===================================================================
print("--- [CNN-LSTM] Xây dựng kiến trúc mô hình ---")
model_cnn_lstm = Sequential([
    # 1. Lớp Embedding: Biến số nguyên thành vector dày đặc
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LENGTH),
    Dropout(0.3),

    # 2. Lớp Conv1D: Trích xuất các đặc trưng cục bộ (n-grams)
    Conv1D(filters=64, kernel_size=5, activation='relu'),

    # 3. Lớp MaxPooling1D: Giảm chiều dài chuỗi, giữ lại đặc trưng quan trọng
    MaxPooling1D(pool_size=4),

    # 4. Lớp LSTM: Học các mối quan hệ tuần tự từ các đặc trưng đã được trích xuất
    Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3)),

    # 5. Lớp Dense (Fully Connected) và Output
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model_cnn_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_cnn_lstm.summary()
print("-" * 30)

--- [CNN-LSTM] Xây dựng kiến trúc mô hình ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

------------------------------


In [23]:
# ===================================================================
# BƯỚC 4: HUẤN LUYỆN MÔ HÌNH
# ===================================================================
print("--- [CNN-LSTM] Bắt đầu huấn luyện ---")
history_cnn_lstm = model_cnn_lstm.fit(
    X_train_pad, y_train_cat,
    epochs=NUM_EPOCHS,
    batch_size=BATCH_SIZE,
    validation_data=(X_test_pad, y_test_cat),
    verbose=2
)
print("Hoàn thành huấn luyện.")
print("-" * 30)
print("--- [CNN-LSTM] Đánh giá trên tập Test ---")
y_pred_prob = model_cnn_lstm.predict(X_test_pad)
y_pred = np.argmax(y_pred_prob, axis=1)

print("\nClassification Report (CNN-LSTM):\n")
print(classification_report(y_test, y_pred, target_names=label_map.keys()))
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy của CNN-LSTM trên tập Test: {accuracy:.4f}")
print("-" * 30)

# print("--- [CNN-LSTM] Lưu mô hình và tokenizer ---")
# # Hãy đảm bảo thư mục MoHinhPhoBERT tồn tại trên Drive của bạn
# model_cnn_lstm.save('/content/drive/MyDrive/MoHinhPhoBERT/cnn_lstm_sentiment_model.h5')
# with open('/content/drive/MyDrive/MoHinhPhoBERT/cnn_lstm_tokenizer.pkl', 'wb') as handle:
#     pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
# print("Đã lưu thành công!")

--- [CNN-LSTM] Bắt đầu huấn luyện ---
Epoch 1/10
2696/2696 - 410s - 152ms/step - accuracy: 0.8121 - loss: 0.5127 - val_accuracy: 0.8594 - val_loss: 0.3955
Epoch 2/10
2696/2696 - 466s - 173ms/step - accuracy: 0.8750 - loss: 0.3606 - val_accuracy: 0.8924 - val_loss: 0.3147
Epoch 3/10
2696/2696 - 417s - 155ms/step - accuracy: 0.9012 - loss: 0.2885 - val_accuracy: 0.9124 - val_loss: 0.2674
Epoch 4/10
2696/2696 - 413s - 153ms/step - accuracy: 0.9160 - loss: 0.2484 - val_accuracy: 0.9258 - val_loss: 0.2329
Epoch 5/10
2696/2696 - 415s - 154ms/step - accuracy: 0.9247 - loss: 0.2199 - val_accuracy: 0.9344 - val_loss: 0.2164
Epoch 6/10
2696/2696 - 438s - 162ms/step - accuracy: 0.9339 - loss: 0.1951 - val_accuracy: 0.9362 - val_loss: 0.2028
Epoch 7/10
2696/2696 - 412s - 153ms/step - accuracy: 0.9395 - loss: 0.1784 - val_accuracy: 0.9446 - val_loss: 0.1929
Epoch 8/10
2696/2696 - 412s - 153ms/step - accuracy: 0.9447 - loss: 0.1631 - val_accuracy: 0.9463 - val_loss: 0.1953
Epoch 9/10
2696/2696 - 442